# Subdivide point clouds into different regions

## Import lib

In [150]:
# from rdflib import Graph, URIRef, Literal, RDF
import open3d as o3d
import os
from pathlib import Path
import numpy as np
import laspy
import cv2
import copy
import ezdxf
from ezdxf.groupby import groupby
import itertools
import math
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.utils import cadutils as cadu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

In [151]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
%autoreload 2

## Inputs

In [153]:
pcd_paths=["D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_elia_equipment.las",
           "D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_elia_terrein.las",
           "D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_elia_vegetatie.las"]
pcd_path="D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_elia_all.las"
pcd_results_path='D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_results.las'
pcd_path_small='D:/Data/2023-03 Elia Substation/Eval_small/pcd.las'
# size=10#m
output_dir='D:/Data/2023-03 Elia Substation/PCD/subclouds2/'
dxf_path='D:/Data/2023-03 Elia Substation/DWG3/cad_autocad_correect_scale.dxf'
# dxf_path='D:/Data/2023-03 Elia Substation/DWG2/cad2.dxf'
blocks_dir='D:/Data/2023-03 Elia Substation/BLOCKS'
cad_dir='D:/Data/2023-03 Elia Substation/BLOCKS_ply'
num_classes=14
transform=np.array([[1,0,0,-122700.00],[0,1,0,-199600.00],[0,0,1,0],[0,0,0,1]])

In [154]:
#subsectie
ground_truth_built=[0,2,4,5,6,7,9,10,11,12,13,14,15,16,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,64,65,66,68,69,70,72,74]
ground_truth_not_built=[1,3,8,17,18,25,26,36,59,60,61,62,63,67,71,73]

In [155]:
pcd_results_path='D:/Data/2023-03 Elia Substation/PCD/lokeren_classificatie_result_final.las'

## Process point clouds as o3d

merge point clouds

In [156]:
pcds=[]
for p in pcd_paths: 
    las  = laspy.read(p)
    pcd = gmu.las_to_pcd(las)
    pcds.append(gmu.color_by_intensity(pcd,las['intensity']))
    
joined_pcd=gmu.join_geometries(pcds)

KeyboardInterrupt: 

In [ ]:
o3d.visualization.draw_geometries([joined_pcd])


In [ ]:
test=copy.deepcopy(joined_pcd)

create octree

In [ ]:
octree = o3d.geometry.Octree(max_depth=4)
octree.convert_from_point_cloud(test, size_expand=0.01)

In [ ]:
octree.traverse(get_indices_per_octree_node)

            3: Leaf node at depth 3 has 178654 points with origin [ 1.22721987e+05  1.99577959e+05 -1.06033570e+01]
            7: Leaf node at depth 3 has 2387 points with origin [1.22721987e+05 1.99577959e+05 7.41971381e+00]
            0: Leaf node at depth 3 has 401151 points with origin [ 1.22740010e+05  1.99559936e+05 -1.06033570e+01]
            1: Leaf node at depth 3 has 109424 points with origin [ 1.22758033e+05  1.99559936e+05 -1.06033570e+01]
            2: Leaf node at depth 3 has 1619090 points with origin [ 1.22740010e+05  1.99577959e+05 -1.06033570e+01]
            3: Leaf node at depth 3 has 1096213 points with origin [ 1.22758033e+05  1.99577959e+05 -1.06033570e+01]
            4: Leaf node at depth 3 has 4422 points with origin [1.22740010e+05 1.99559936e+05 7.41971381e+00]
            5: Leaf node at depth 3 has 1463 points with origin [1.22758033e+05 1.99559936e+05 7.41971381e+00]
            6: Leaf node at depth 3 has 79354 points with origin [1.22740010e+05 1.99

traverse and export data

In [ ]:
import uuid

lowEnd=150000
highEnd=1500000
a=len(octree.root_node.children)
a=-1
for node in octree.root_node.children[0:1]: # depth1
    a+=1
    count=len(node.indices) if getattr(node,'indices',None) is not None else 0
    if (getattr(node,'indices',None) is not None and 
        len(node.indices)> lowEnd and 
        len(node.indices)< highEnd):
        o3d.io.write_point_cloud( os.path.join(output_dir,f'pcd_{a}.pcd'),joined_pcd.select_by_index(node.indices))
       
    else:
        # b=len(getattr(node,'children',[]))
        b=-1
        for childnode in getattr(node,'children',[]): #depth 2
            b+=1
            # count=len(childnode.indices) if getattr(childnode,'indices',None) is not None else 0
            if (getattr(childnode,'indices',None) is not None and 
                len(childnode.indices)> lowEnd and 
                len(childnode.indices)< highEnd):
                o3d.io.write_point_cloud( os.path.join(output_dir,f'pcd_{a}_{b}.pcd'),joined_pcd.select_by_index(childnode.indices))
                np.save(os.path.join(output_dir,f'pcd_{a}_{b}.npy'),)
                
                pcd=joined_pcd.select_by_index(childnode.indices)
                points=np.asarray(pcd.points)
                feat=np.asarray(pcd.colors)
                labels=labelarray[childnode.indices]
                # data = {'point': points, 'feat': feat, 'label': labels}
                data=np.vstack((points,labels,feat))
            else:
                # c=len(getattr(childnode,'children',[]))
                c=-1
                for grandchildnode in getattr(childnode,'children',[]): #depth 3
                    c+=1
                    # count=len(grandchildnode.indices) if getattr(grandchildnode,'indices',None) is not None else 0
                    if (getattr(grandchildnode,'indices',None) is not None and 
                        len(grandchildnode.indices)> lowEnd and 
                        len(grandchildnode.indices)< highEnd):
                    # if count> lowEnd and count<highEnd:
                        o3d.io.write_point_cloud( os.path.join(output_dir,f'pcd_{a}_{b}_{c}.pcd'),joined_pcd.select_by_index(grandchildnode.indices))
                        
                    else:
                        # d=len(getattr(grandchildnode,'children',[]))
                        d=-1
                        for grandgrandchildnode in getattr(grandchildnode,'children',[]): #depth 4
                            d+=1
                            if (getattr(grandgrandchildnode,'indices',None) is not None and 
                                len(grandgrandchildnode.indices)> lowEnd and 
                                len(grandgrandchildnode.indices)< highEnd):
                            # count=len(grandgrandchildnode.indices) if getattr(grandgrandchildnode,'indices',None) is not None else 0
                            # if count> lowEnd and count<highEnd:
                                o3d.io.write_point_cloud( os.path.join(output_dir,f'pcd_{a}_{b}_{c}_{d}.pcd'),joined_pcd.select_by_index(grandgrandchildnode.indices))
                                


In [ ]:
def get_indices_per_octree_node(node, node_info):
    lowEnd=100000
    highEnd=500000
    if isinstance(node, o3d.geometry.OctreePointColorLeafNode):
        print("{}{}: Leaf node at depth {} has {} points with origin {}".
                  format('    ' * node_info.depth, node_info.child_index,
                         node_info.depth, len(node.indices), node_info.origin))
        count=len(node.indices)
        if count>lowEnd and count<highEnd:
            o3d.io.write_point_cloud( os.path.join(output_dir,f'pcd_{str(uuid.uuid1())}.pcd') ,joined_pcd.select_by_index(node.indices))

## Process point clouds as las files

import las file

In [ ]:
las  = laspy.read(pcd_path)

In [ ]:
print(list(las.point_format.dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue']


In [ ]:
points = np.array(las.xyz, dtype=np.float32)
feat = np.array([las['intensity']], dtype=np.int32).T
labels = np.array([las['classification']], dtype=np.int32).T
data=np.hstack((points,labels,feat))

create point cloud

In [ ]:
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(las.xyz)
pcd=gmu.color_by_intensity(pcd,las['intensity'])
print(len(np.asarray(pcd.points)))

86580621


create octree

In [ ]:
octree = o3d.geometry.Octree(max_depth=4)
octree.convert_from_point_cloud(test, size_expand=0.01)

traverse and gather data

In [ ]:
idxLists,pathLists= pt.subdivide_pcd_per_octree(pcd,maxDepth=4,lowEnd=0,highEnd=2000000)
print(len(pathLists))

23


In [ ]:
print(f'number of paths:{len(pathLists)}')

In [ ]:
lowEnd=0
highEnd=2000000
a=-1
pathLists=[]
idxLists=[]
for node in octree.root_node.children[0:1]: # depth1
    a+=1
    if (getattr(node,'indices',None) is not None and 
        len(node.indices)> lowEnd and 
        len(node.indices)< highEnd):
        pathLists.append(os.path.join(output_dir,f'pcd_{a}.npy'))
        idxLists.append(node.indices)
       
    else:
        b=-1
        for childnode in getattr(node,'children',[]): #depth 2
            b+=1
            if (getattr(childnode,'indices',None) is not None and 
                len(childnode.indices)> lowEnd and 
                len(childnode.indices)< highEnd):
                pathLists.append(os.path.join(output_dir,f'pcd_{a}_{b}.npy'))
                idxLists.append(childnode.indices)
        
            else:
                c=-1
                for grandchildnode in getattr(childnode,'children',[]): #depth 3
                    c+=1
                    if (getattr(grandchildnode,'indices',None) is not None and 
                        len(grandchildnode.indices)> lowEnd and 
                        len(grandchildnode.indices)< highEnd):
                        pathLists.append(os.path.join(output_dir,f'pcd_{a}_{b}_{c}.npy'))
                        idxLists.append(grandchildnode.indices)
                    else:
                        d=-1
                        for grandgrandchildnode in getattr(grandchildnode,'children',[]): #depth 4
                            d+=1
                            if (getattr(grandgrandchildnode,'indices',None) is not None and 
                                len(grandgrandchildnode.indices)> lowEnd and 
                                len(grandgrandchildnode.indices)< highEnd):
                                pathLists.append(os.path.join(output_dir,f'pcd_{a}_{b}_{c}_{d}.npy'))
                                idxLists.append(grandgrandchildnode.indices)


number of paths:23


In [ ]:
print(pathLists)

['D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_4.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_0.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_1.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_2_4.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_2_5.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_2_6.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_2_7.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_3_4.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_3_5.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_3_6.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_3_7.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_5_7.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_6_1_4.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_6_1_5.npy', 'D:/Data/2023-03 Elia Substation/PCD/subclouds/pcd_0_6_1_6.npy', 'D:/Data/2023-03 Elia Substation/P

gather data and export

In [ ]:
for p,indices in zip(pathLists,idxLists):
    np.save(p,data[indices])

## Process point clouds with fixed regions

In [ ]:
las  = laspy.read(pcd_path)

In [ ]:
array,names=gmu.las_get_data(las)
print(names)

['X', 'Y', 'Z', 'red', 'green', 'blue', 'intensity', 'classification']


In [ ]:
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(las.xyz)
print(len(np.asarray(pcd.points)))

86580621


In [ ]:
pcd=gmu.color_by_intensity(pcd,las['intensity'])

In [ ]:
# pcd=pcd.random_down_sample(0.1)

subdivide space

In [ ]:
size=[10,100,100]#m
parts=[7,7,1]
box=pcd.get_axis_aligned_bounding_box()
boxes,names=gmu.divide_box_in_boxes(box,parts=parts)
for box in boxes:
    box.color=[1,0,0]
print(f' number of boxes: {len(boxes)}')

 number of boxes: 49


In [ ]:
# create boxes
o3d.visualization.draw_geometries(boxes[0:15]+[pcd])

select point clouds

In [ ]:
idxLists,names=pt.subdivide_pcd_per_box(pcd,parts=parts)

In [ ]:
for idx,name in zip(idxLists,names):
    if idx:
        data,_=gmu.las_get_data(las,indices=idx)
        print(data[:,0:3].shape,data[:,7].shape,data[:,6].shape)
        train_data=np.hstack((data[:,0:3],np.array([data[:,7]]).T,np.array([data[:,6]]).T))
        train_dir=os.path.join(output_dir,name+'.npy')
        np.save(train_dir, train_data)
        # pcdtest=pcd.select_by_index(idx)

(29658, 3) (29658,) (29658,)
(1302618, 3) (1302618,) (1302618,)
(218999, 3) (218999,) (218999,)
(1608, 3) (1608,) (1608,)
(1366239, 3) (1366239,) (1366239,)
(1303602, 3) (1303602,) (1303602,)
(3209538, 3) (3209538,) (3209538,)
(2760910, 3) (2760910,) (2760910,)
(14798, 3) (14798,) (14798,)
(209227, 3) (209227,) (209227,)
(2871610, 3) (2871610,) (2871610,)
(2175844, 3) (2175844,) (2175844,)
(1642834, 3) (1642834,) (1642834,)
(1838064, 3) (1838064,) (1838064,)
(4128509, 3) (4128509,) (4128509,)
(47589, 3) (47589,) (47589,)
(881204, 3) (881204,) (881204,)
(2521304, 3) (2521304,) (2521304,)
(3045488, 3) (3045488,) (3045488,)
(2415779, 3) (2415779,) (2415779,)
(2137054, 3) (2137054,) (2137054,)
(1837935, 3) (1837935,) (1837935,)
(2447464, 3) (2447464,) (2447464,)
(1483865, 3) (1483865,) (1483865,)
(2491931, 3) (2491931,) (2491931,)
(1761270, 3) (1761270,) (1761270,)
(4319148, 3) (4319148,) (4319148,)
(7523220, 3) (7523220,) (7523220,)
(171218, 3) (171218,) (171218,)
(4875449, 3) (4875449,) 

In [ ]:
inliers_pcd = pcd.select_by_index(inliers_indices[1], invert=False) # select inside points = cropped 
print(inliers_pcd)

PointCloud with 31067 points.


In [ ]:
array=gmu.las_get_data(las,idxLists[0:1])


In [ ]:
las_sub=gmu.pcd_to_dataframe()


In [ ]:
df=gmu.las_to_dataframe(pcd)

In [ ]:
describe_element('test',)

In [ ]:
o3d.visualization.draw_geometries(boxes[0:30]+[pcdtest])

# load seperate files to check validity

In [ ]:
data=np.load(os.path.join(output_dir,'pcd_0_3_0.npy'))

In [ ]:
pcd=o3d.geometry.PointCloud(o3d.utility.Vector3dVector(data[:,0:3]))

pcd=gmu.color_by_intensity(pcd,data[:,0:4])
print(len(np.asarray(pcd.points)))

RuntimeError: 

In [ ]:
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(data[:,0:3])
print(len(np.asarray(pcd.points)))

881204


In [ ]:
pcd.colors=o3d.utility.Vector3dVector(gmu.array_to_colors(data[:,3]))

In [ ]:
print (pcd)

PointCloud with 881204 points.


In [ ]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


# Parse DXF

load data

In [ ]:
import ezdxf

#open dxf file
dwg = ezdxf.readfile(dxf_path)

#select layout: Modelspace,Paperspace,Blocklayout
msp = dwg.modelspace()


print info

In [ ]:
#layers
print([key for key in dwg.layers.entries])

['0', 'structure', 'fondation', 'flange', 'equipement', 'equipement_10', 'earthing', 'earthing_17', 'level 14', 'level 27', 'level 13', 'flange_16', 'level 4', 'level 50', 'level 3', 'structure_11', 'cable_naked_14', 'fondation_5', 'tube_15', 'level 2', 'malt', 'various', 'dimension_50', 'level 63', 'level 16', 'level 11', 'level 35', 'level 15', 'level 8', 'am_0', 'level 1', 'level 7', 'raccords', 'cotations', 'charpente', 'cable_18', 'noise_barriers_35', 'gutter_6', 'level 12', 'level 6', 'level 5', 'level 17', 'level 18', 'level 10', 'level 40', 'cable', 'roof_36', 'grates_38', 'cable_naked_14_bis', 'porte  pour batiment', 'escalier beton', 'border_1', 'level 9', 'fondations', 'hatching_29', 'safety_lighting_24', 'toit', 'sewage_7', 'text_fr_40', 'text_nl_41', 'building_30', 'permits_text_43', 'level 57', 'dimension', 'tube', 'gutter', 'armoire travee', 'level 41', 'various_27', 'eclairage poste', 'point accrochage', 'tiles_33', 'fondation future', 'route fill', 'colonne logette tfo

convert data

In [ ]:
geometry_groups,layers=cadu.ezdxf_to_o3d(dwg,
                                    # dtypes=['LINE','POLYLINE'],
                                    # layers=['0'],
                    join_geometries=True,
                    explode_blocks=False) 
print(len(geometry_groups),len(layers))
print(geometry_groups[0:5])
print(layers[0:5])

77 layers found.
29821 entities found.
Converting entities to open3d geometries...


d:\Scan-to-BIM repository\geomapi\geomapi\utils\cadutils.py:553: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  faces=gmu.split_quad_faces(np.array(entity.faces))


RuntimeError: 

In [ ]:
dtypes=[]
for entity in msp:
    dtypes.append(entity.dxftype())
    # if entity.dxftype() == 'LINE':
    #     None
    #     #    print(entity)
    # if entity.dxftype() == 'ARC':
    #     None
    #     # print(entity)
    # if entity.dxftype() == 'CIRCLE':
    #     None
    #     # print(entity)
    # if entity.dxftype() == 'POINT':
    #     None
    #     # print(entity)
    # if entity.dxftype() == 'SPLINE':
    #     None
    # #    print(entity)
    # if entity.dxftype() == 'POLYLINE':
    #     None
    #     print(entity)
    # if entity.dxftype() == 'LWPOLYLINE':
    #     None
    #     print(entity)
    # if entity.dxftype() == 'MESH':        
    #     print(entity)
    # if entity.dxftype() == 'ELLIPSE':
    #     None
        # print(entity)
    # elif entity.dxftype() == 'HATCH':
    #     o3d_geometries.append(hatch_to_o3d(entity))
    # if entity.dxftype() == 'SOLID or 3DFACE':
    #    print(entity)
    # else:
    #     None
        # print(entity)
array=np.array(dtypes)
unique,counts= np.unique(array,return_counts=True)
print(unique, counts)

['3DFACE' '3DSOLID' 'ARC' 'CIRCLE' 'DIMENSION' 'ELLIPSE' 'HATCH' 'INSERT'
 'LEADER' 'LINE' 'LWPOLYLINE' 'MTEXT' 'POINT' 'POLYLINE' 'REGION' 'SOLID'
 'SPLINE' 'SURFACE' 'TEXT'] [  329  1873   423   366     2    28    59  1074     2 24051  2665     8
   139  1611    42    31   178    14    30]


In [ ]:
# linecounter=0
# meshcounter=0
# pcdcounter=0
# for g in list(itertools.chain(*geometry_groups)):
#     if type(g)==o3d.geometry.LineSet:
#         linecounter+=1
#     if type(g)==o3d.geometry.TriangleMesh:
#         meshcounter+=1
#     if 'cloud' in str(type(g)):
#         pcdcounter+=1
# print(linecounter,meshcounter,pcdcounter)

129027 1567 0


filter far away entities

In [ ]:
geometries=geometry_groups
center_point=np.array([122697.25,199620.05,5])
geometries=gmu.filter_geometry_by_distance(geometries,center_point,distance_threshold=500)
print(geometries)

[PointCloud with 12 points., LineSet with 498 lines., PointCloud with 22 points., PointCloud with 1 points., LineSet with 897 lines., PointCloud with 1 points., PointCloud with 89 points., PointCloud with 1 points., PointCloud with 3 points., LineSet with 211 lines., LineSet with 2636 lines., PointCloud with 6 points., LineSet with 122 lines., PointCloud with 4 points., LineSet with 72 lines.]


visualize

In [ ]:
# testgeometries=geometries
# for geom in testgeometries:
[g.paint_uniform_color(ut.random_color()) for g in geometries]
# test=[g.paint_uniform_color([1,0,0]) for g in list(itertools.chain(*testgeometries))]
# test=gmu.join_geometries(geometries)
# flat_list = list(itertools.chain(*testgeometries))
o3d.visualization.draw_geometries(geometries)  

In [ ]:
testgeometries=geometries
for geom in testgeometries:
    [g.paint_uniform_color(ut.random_color()) for g in ut.item_to_list(geom)]
# test=[g.paint_uniform_color([1,0,0]) for g in list(itertools.chain(*testgeometries))]
# test=gmu.join_geometries(geometries)
flat_list = list(itertools.chain(*testgeometries))
o3d.visualization.draw_geometries(flat_list)  

access blocks

In [ ]:
test=o3d.io.read_triangle_mesh('D:/Data/2023-03 Elia Substation/BLOCKS/ARE STA170kV 40kA Cde tri manuelle.ply')
o3d.visualization.draw_geometries([test])  

In [ ]:
import glob, os

# 0. get block definitions

# get all blocks_refs
block_refs=list(msp.query("INSERT"))


# select unique blocks that occur in block_refs and do have an internal *
block_refs=[b for b in block_refs if b.dxf.name[0] != "*"]
names,indices=np.unique(np.array([b.dxf.name for b in block_refs]),return_index=True)
print(names)

#get exterior blocks if exists
open3d_blocks={}
if os.path.exists(blocks_dir):
    files=ut.get_list_of_files(blocks_dir)
    for f in files:
        open3d_blocks[ut.get_filename(f)]=ut.item_to_list(o3d.io.read_triangle_mesh(f))


# convert block geometries to open3d geometries
for block_ref in np.array(block_refs)[indices[0:1]]:  
    #get geometries from seperate folder
    if block_ref.dxf.name in list(open3d_blocks.keys()):
        continue
    else:
    # else get geometries directly from dxf (this is imperfect)
        geometries,_=cadu.ezdxf_entities_to_o3d([e for e in block_ref.virtual_entities()])
        open3d_blocks[block_ref.dxf.name]=ut.item_to_list(geometries)
        # print(block_ref.dxf.name)
        # print(entities)
    
 
print(open3d_blocks)          

['ARE STA170kV 40kA Cde tri manuelle' 'ART CA170 B1-B4-B5-B6'
 'ART CA170 B1-B4-B5-B6_1' 'ART CA170 B1-B4-B5-B6_2' 'ART UTE 170'
 'ART UTE 170_1' 'ART UTE 170_2' 'BC01' 'BOUCHON 1' 'BOUCHON 2'
 'BOUCHON 2_1' 'CAPTEUR MALT 6' 'CAPTEUR MALT 6_1' 'CAPTEUR MALT 8'
 'CHAIN4' 'CHAIN4_1' 'CHAIN4_2' 'DILATATION 3' 'DILATATION 5' 'From Solid'
 'GADE CORPS' 'GADE CORPS_1' 'HE300A' 'HEB220'
 'ISO C16 750 III Posé (3D)_3' 'ISO C8 750  III (3D)' 'ISO02' 'ISO06'
 'ISO5' 'MALT PIECE W' 'MALT PIECE W 1 BOULE' 'MALT02' 'PAR05'
 'PAU TFO PN 30kV 2000A-5sec' 'PLAT_D' 'POINT.DIAM6' 'PORTE 1'
 'PORTE+IMPOSTE' 'PORTIQUE JEU DE BARRES (2010)_1' 'RAC DROIT 4' 'RAC T 2'
 'RAC T 36' 'RAC T 36_1' 'RAC004' 'RAC010' 'RAC010_1' 'RAC022' 'RAC025'
 'RAC025_1' 'RAC03' 'RAC031' 'RAC031_1' 'RAC034' 'RAC03_1' 'RAC043'
 'RAC043_1' 'RAC043_2' 'RAC043_3' 'RAC043_4' 'RAC043_5' 'RAC043_6'
 'RAC050' 'RAC050_1' 'RAC051' 'RAC057' 'RAC057_1' 'RAC058' 'RAC089'
 'RAC090' 'RAC091' 'RAC093' 'RAC093_1' 'RAC093_2' 'RAC094' 'RAC094_1'
 

In [ ]:
n=1
show=open3d_blocks.get('ARE STA170kV 40kA Cde tri manuelle')
print(names[n])
print(show)

geo=[s for s in show if type(s)==o3d.geometry.LineSet or type(s)==o3d.geometry.TriangleMesh]
o3d.visualization.draw_geometries(geo)  

ART CA170 B1-B4-B5-B6
[LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 l

In [ ]:
from scipy.spatial.transform import Rotation as R         
# fetch and place blocks in correct places
for block_ref in block_refs:
    if block_ref.dxf.name in list(open3d_blocks.keys()):
        #get geometry
        geometries=open3d_blocks['block_ref.dxf.name'].value() 
        #get cartesian transform
        translation=block_ref.dxf.insert
        r = R.from_euler('z', block_ref.dxf.rotation, degrees=True) #degrees
        rotation=r.as_matrix()
        scale=block.dxf.scale
        transform=gmu.get_cartesian_transform(rotation=rotation,
                                    translation=translation,
                                    scale=scale)
        #transform geometry
        geometries=gmu.transform_geometries(geometries,transform) #more general transform function 
        geometry_groups.append(geometries)

In [ ]:
print(f'blocks : {len(msp.query("INSERT"))}')
# for e in msp.query("INSERT"):
#     e.explode()
    
print(f'lines : {len(msp.query("LINE"))}') # cadu.
print(cadu.lines_to_o3d(msp.query("LINE")[0:3]))

print(f'arcs : {len(msp.query("ARC"))}')
print(cadu.arcs_to_o3d(msp.query("ARC")[0:3]))

print(f'points : {len(msp.query("POINT"))}')
print(cadu.points_to_o3d(msp.query("POINT")[0:3]))

print(f'circle : {len(msp.query("CIRCLE"))}')
print(cadu.circles_to_o3d(msp.query("CIRCLE")[0:3]))

print(f'spline : {len(msp.query("SPLINE"))}')
print(cadu.splines_to_o3d(msp.query("SPLINE")[0:3]))

print(f'polyline : {len(msp.query("POLYLINE"))}')
print(cadu.polylines_to_o3d(msp.query("POLYLINE")[0:3]))

# print(f'mesh : {len(msp.query("MESH"))}')

print(f'elipse : {len(msp.query("ELLIPSE"))}')
print(cadu.ellipses_to_o3d(msp.query("ELLIPSE")[0:3]))

# print(f'hatch : {len(msp.query("HATCH"))}') #! is this necessary
# print(cadu.hatches_to_o3d(msp.query("HATCH")[0:3]))

print(f'solids : {len(msp.query("SOLID or 3DFACE"))}')
print(cadu.solids_to_o3d(msp.query("SOLID")[0:3]))

print(f'solids : {len(msp.query("3DSOLID"))}')
print(cadu.solids_to_o3d(msp.query("3DSOLID")[0:3]))


blocks : 1074
lines : 24051
([LineSet with 1 lines., LineSet with 1 lines., LineSet with 1 lines.], ['0', '0', '0'])
arcs : 423
([LineSet with 9 lines., LineSet with 9 lines., LineSet with 9 lines.], ['Level 4', 'Level 1', 'Level 12'])
points : 139
(array([[1.22772566e+05, 1.99649039e+05, 5.01728683e+00],
       [1.22735858e+05, 1.99650773e+05, 9.79696529e+00],
       [1.22708818e+05, 1.99666051e+05, 5.03729683e+00]]), ['POINT ACCROCHAGE', 'RACCORDS', '0'])
circle : 366
([LineSet with 35 lines., LineSet with 35 lines., LineSet with 35 lines.], ['Level 7', 'Level 7', 'Level 7'])
spline : 178
([LineSet with 3 lines., LineSet with 3 lines., LineSet with 3 lines.], ['CABLE_NAKED_14', 'CABLE_NAKED_14', 'CABLE_NAKED_14'])
polyline : 1611
([TriangleMesh with 24 points and 12 triangles., TriangleMesh with 48 points and 20 triangles., TriangleMesh with 48 points and 20 triangles.], ['FONDATION', 'STRUCTURE', 'STRUCTURE'])
elipse : 28
([LineSet with 9 lines., LineSet with 9 lines., LineSet with 

DXFAttributeError: Invalid DXF attribute "vtx0" for entity 3DSOLID

sample geometries

In [ ]:
sampled_geometries=gmu.sample_geometry(geometries[0][0])
print(sampled_geometries)

[PointCloud with 40959324 points.]


In [ ]:
testgeometries=[geometries[0][0]]
test=[g.paint_uniform_color([1,0,0]) for g in testgeometries]
# test=gmu.join_geometries(geometries)
o3d.visualization.draw_geometries(test)  

# CAD vs PCD validation

## Prepare

load cad objects

In [ ]:
cad_dir=os.path.join('D:/Data/2023-03 Elia Substation/BLOCKS_test/')


In [ ]:
mesh=o3d.io.read_triangle_mesh('D:/Data/2023-03 Elia Substation/BLOCKS_test/1f565f6b-1d73-4ef2-b69a-019c150e7d93.ply')
print(mesh)
# import trimesh
# mesh = trimesh.load_mesh('D:/Data/2023-03 Elia Substation/BLOCKS_ply/9_Isolator/0a8c339b-f2f6-487c-84d1-2c28000f9071.ply')

[Open3D WARNING] Read PLY failed: unable to open file: D:/Data/2023-03 Elia Substation/BLOCKS_test/1f565f6b-1d73-4ef2-b69a-019c150e7d93.ply
TriangleMesh with 0 points and 0 triangles.


In [ ]:
cadNodes= []
if os.path.exists(cad_dir):
    files=ut.get_list_of_files(cad_dir)
    for f in files:
        mesh=o3d.io.read_triangle_mesh(f)
        if len(np.asarray(mesh.triangles)) >0:
            #put it locally
            mesh.transform(transform) 
            cadNodes.append(MeshNode(name=ut.get_filename(f),
                                resource=mesh,
                                layer=ut.get_folder(f)))
            
#sample point cloud
resources=[n.resource for n in cadNodes if n.resource]
pcd_cad,identity_array=gmu.create_identity_point_cloud(resources,resolution=0.01)            
[n.resource.paint_uniform_color(ut.random_color()) for n in cadNodes if n.resource]            
joined_o3d_objects=gmu.join_geometries(resources)
print(f' {len(cadNodes)} objects loaded.')

[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNIN

load pcd

In [ ]:
las  = laspy.read(pcd_results_path)
pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las,transform=transform,getColors=True))

visualize

In [ ]:
testgeometries=resources
[g.paint_uniform_color(ut.random_color()) for g in testgeometries]
o3d.visualization.draw_geometries([pcd_cad])  

In [ ]:
#create ground truth
# for i,g in enumerate(open3d_objects):
#     print(i)
#     o3d.visualization.draw_geometries([pcdNode.resource,g.paint_uniform_color([1,0,0])])

In [ ]:
t1=[g.resource.paint_uniform_color([0.5,0.5,0.5]) for g in cadNodes]
t2=[g.resource.paint_uniform_color([1,0,0]) for g in nodes]
o3d.visualization.draw_geometries(t1+t2)  

locally register pcd_cad and pcdNode

In [ ]:
registration = o3d.pipelines.registration.registration_icp(
        pcd_cad, pcdNode.resource, 0.1)
print(registration)
print(registration.transformation)

RegistrationResult with fitness=5.587643e-01, inlier_rmse=3.489094e-02, and correspondence_set size of 805157
Access transformation to get result.
[[ 9.99988546e-01  5.73187597e-05  4.78587919e-03 -1.20654332e-01]
 [-6.51379449e-05  9.99998663e-01  1.63366341e-03 -1.26285267e-01]
 [-4.78577915e-03 -1.63395644e-03  9.99987213e-01  4.71851033e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
pcd_cad_t=copy.deepcopy(pcd_cad)
pcd_cad_t.transform(registration.transformation)
[g.transform(registration.transformation) for g in open3d_objects]

[TriangleMesh with 2298 points and 3368 triangles.,
 TriangleMesh with 465 points and 706 triangles.,
 TriangleMesh with 1189 points and 1630 triangles.,
 TriangleMesh with 1933 points and 2926 triangles.,
 TriangleMesh with 624 points and 1116 triangles.,
 TriangleMesh with 430 points and 600 triangles.,
 TriangleMesh with 8825 points and 13278 triangles.,
 TriangleMesh with 152601 points and 247584 triangles.,
 TriangleMesh with 453 points and 690 triangles.,
 TriangleMesh with 615 points and 1119 triangles.,
 TriangleMesh with 1020 points and 1556 triangles.,
 TriangleMesh with 298 points and 438 triangles.,
 TriangleMesh with 291 points and 424 triangles.,
 TriangleMesh with 608 points and 942 triangles.,
 TriangleMesh with 858 points and 1300 triangles.,
 TriangleMesh with 1139 points and 1608 triangles.,
 TriangleMesh with 41 points and 46 triangles.,
 TriangleMesh with 696 points and 952 triangles.,
 TriangleMesh with 447 points and 666 triangles.,
 TriangleMesh with 662 points 

In [ ]:
o3d.io.write_point_cloud('D:/Data/2023-03 Elia Substation/Eval_small/pcd_cad_t.ply',pcd_cad_t)

True

In [ ]:
#visualize transformed pcds
o3d.visualization.draw_geometries([pcd_cad_t,pcdNode.resource])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


## Evaluate which objects are not in the point cloud i.e. which objects do no have nearby points


option A: mark objects that do not have pcd points as their closest points 

In [ ]:
#compute distance from pcd to cad
indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcdNode.resource.points),np.asarray(pcd_cad_t.points))

In [ ]:
#filter distances
distance_threshold=0.3
distances=distances[:,0]
inlier_indices=indices[distances < distance_threshold]
#get cad indices that lie within that threshold
indices_in_cad=identity_array[inlier_indices]
indices_unique_in_cad=np.unique(indices_in_cad)
#invert the selection
range=np.arange(0,len(open3d_objects))
indices_not_in_cad=[i for i in range if i not in indices_unique_in_cad]
print(indices_not_in_cad)
print(f'Recall: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(ground_truth_not_built)}')
print(f'Precision: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(indices_not_in_cad)}')


[8, 10, 16, 24, 30, 39, 45, 47, 61, 71, 73]
Recall: 0.25
Precision: 0.36363636363636365


In [ ]:
#colorize objects that are not the closest object to the point clouds
[g.paint_uniform_color([1,0,0]) for i,g in enumerate(open3d_objects) if i in indices_not_in_cad ] #red
[g.paint_uniform_color([0,1,0]) for i,g in enumerate(open3d_objects) if i not in indices_not_in_cad ] #green
joined_mesh=gmu.join_geometries(open3d_objects)
o3d.visualization.draw_geometries([pcdNode.resource,joined_mesh])  

option B: mark objects that have at least one point far from the point cloud

In [ ]:
#compute distance from cad to pcd
distance_threshold=0.1
dists = pcd_cad_t.compute_point_cloud_distance(pcdNode.resource)

#get indices of points outside the distance_threshold
ind = np.where(np.asarray(dists) > distance_threshold)[0]
indices_not_in_cad=np.unique(identity_array[ind])
print(indices_not_in_cad)
print(f'Recall: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(ground_truth_not_built)}')
print(f'Precision: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(indices_not_in_cad)}')

[0 3 6 7 8 17 21 23 33 35 37 38 41 44 49 53 59 61 63 67 70 73]
Recall: 0.5
Precision: 0.36363636363636365


In [ ]:
#colorize objects that have points further tha the distance_threshold from the pcd
[g.paint_uniform_color([1,0,0]) for i,g in enumerate(open3d_objects) if i in indices_not_in_cad ] #red
[g.paint_uniform_color([0,1,0]) for i,g in enumerate(open3d_objects) if i not in indices_not_in_cad ] #green
joined_mesh=gmu.join_geometries(open3d_objects)
o3d.visualization.draw_geometries([pcdNode.resource,joined_mesh])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


option C: mark objects with not atleast 70% of points near pcd

In [ ]:
#compute distance from cad to pcd
distance_threshold=0.1
percentage_threshold=0.7
dists = pcd_cad_t.compute_point_cloud_distance(pcdNode.resource)

#get indices of points outside the distance_threshold
ind = np.where(np.asarray(dists) <= distance_threshold)[0]
indices_unique_in_cad,counts=np.unique(identity_array[ind],return_counts=True)
unique_totals,count_totals=np.unique(identity_array,return_counts=True)
indices_in_cad=[]
for i,c in zip(indices_unique_in_cad,counts):
    if c/count_totals[i]>percentage_threshold:
        indices_in_cad.append(i)
#invert the selection
range=np.arange(0,len(open3d_objects))
indices_not_in_cad=[i for i in range if i not in indices_in_cad]
print(indices_not_in_cad)
print(f'Recall: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(ground_truth_not_built)}')
print(f'Precision: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(indices_not_in_cad)}')

[0, 6, 35, 44, 49, 63, 67, 70]
Recall: 0.125
Precision: 0.25


In [ ]:
#colorize objects that have points further tha the distance_threshold from the pcd
[g.paint_uniform_color([1,0,0]) for i,g in enumerate(open3d_objects) if i in indices_not_in_cad ] #red
[g.paint_uniform_color([0,1,0]) for i,g in enumerate(open3d_objects) if i not in indices_not_in_cad ] #green
joined_mesh=gmu.join_geometries(open3d_objects)
o3d.visualization.draw_geometries([pcdNode.resource,joined_mesh])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


Enrich analysis with classification results

In [ ]:
#compute indices from cad to pcd and retrieve semantic segmenation results
indices,_=gmu.compute_nearest_neighbors(np.asarray(pcd_cad_t.points),np.asarray(pcdNode.resource.points))
pcd_labels=np.int8(las.label[indices])
pcd_predictions=np.int8(las.prediction[indices])

NameError: name 'pcd_cad_t' is not defined

In [ ]:
# compute classes per object (presence of at least 20%)
object_classes=[] 
for i,o in enumerate(open3d_objects): 
    local_labels=pcd_labels[identity_array==i]
    unique,counts=np.unique(local_labels,return_counts=True)
    object_classes.append(unique[counts/local_labels.size>0.2])

In [ ]:
# split point cloud per prediction
class_point_clouds=[]
for i in np.arange(num_classes): 
    class_point_clouds.append(np.asarray(pcdNode.resource.points)[las.prediction==i] )
class_point_clouds=np.array(class_point_clouds)

C:\Users\u0094523\AppData\Local\Temp/ipykernel_18584/1611104880.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  class_point_clouds=np.array(class_point_clouds)


In [ ]:
distance_threshold=0.1
percentage_threshold=0.7
built=np.zeros(len(open3d_objects))
#per object, group relevant class_point clouds and make the nearest neighbor analysis
for i,o in enumerate(open3d_objects): 
    #get querry points
    query_points=np.asarray(pcd_cad_t.points)[identity_array==i] 
    #get classes
    classes=object_classes[i]
    #get class_point_clouds
    points=class_point_clouds[classes]
    pcd=gmu.join_geometries(points)
    #compute nearest neighbors
    indices,distances=gmu.compute_nearest_neighbors(query_points,np.asarray(pcd.points))
    d=distances[distances<= distance_threshold]
    #assign built status
    if d.size/query_points.shape[0]>=percentage_threshold:
        built[i]=1
print(f'{np.sum(built)} of {len(open3d_objects)} elements are built')
print([i for i,b in enumerate(built) if b==0])
indices_not_in_cad=[i for i,b in enumerate(built) if b==0]
print(f'Recall: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(ground_truth_not_built)}')
print(f'Precision: {len([i for i in indices_not_in_cad if i in ground_truth_not_built]) /len(indices_not_in_cad)}')

52.0 of 75 elements are built
[0, 3, 5, 6, 18, 23, 25, 26, 27, 29, 35, 36, 39, 40, 42, 44, 45, 49, 59, 61, 63, 67, 70]
Recall: 0.5625
Precision: 0.391304347826087


In [ ]:
#colorize objects that are built
[g.paint_uniform_color([1,0,0]) for  i,g in enumerate(open3d_objects) if built[i] ==0] #red
[g.paint_uniform_color([0,1,0]) for i,g in enumerate(open3d_objects) if built[i] !=0] #green
joined_mesh=gmu.join_geometries(open3d_objects)
o3d.visualization.draw_geometries([pcdNode.resource,joined_mesh])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


# CAD vs classified PCD

Evaluate which objects are not drawn i.e. which point cloud points marked as equipment are not near and drawn objects.

In [ ]:
#compute indices from cad to pcd and retrieve semantic segmenation results
indices,_=gmu.compute_nearest_neighbors(np.asarray(pcd_cad_t.points),np.asarray(pcdNode.resource.points))
pcd_labels=np.int8(las.label[indices])
pcd_predictions=np.int8(las.prediction[indices])

In [ ]:
# split point cloud per prediction
pcd_per_class=[]
for i in np.arange(num_classes):    
    ind = np.where(las.prediction==i)[0]
    pcd_per_class.append(pcdNode.resource.select_by_index(ind))
pcd_per_class=np.array(pcd_per_class)
print(pcd_per_class)

[PointCloud with 1248 points. PointCloud with 440 points.
 PointCloud with 0 points. PointCloud with 3764 points.
 PointCloud with 14636 points. PointCloud with 3283 points.
 PointCloud with 1131 points. PointCloud with 69420 points.
 PointCloud with 1718 points. PointCloud with 43300 points.
 PointCloud with 8719 points. PointCloud with 18168 points.
 PointCloud with 270 points. PointCloud with 22505 points.]


In [ ]:
# compute classes per object (presence of at least 20%)
object_classes=[] 
for i,o in enumerate(open3d_objects): 
    local_labels=pcd_labels[identity_array==i]
    unique,counts=np.unique(local_labels,return_counts=True)
    object_classes.append(unique[counts/local_labels.size>0.2])

In [ ]:
# split pcd_Cad_t per label -> pcd_class_objects
pcd_class_objects=[None] * num_classes #point clouds of objects grouped per class
for i in np.arange(num_classes):
    #get all object indices belonging to a class   
    indices=[j for j,o in enumerate(object_classes) if i in o] 
    if len(indices)>0:
        #get points
       pcd_class_objects[i]=gmu.join_geometries([np.asarray(pcd_cad_t.points)[identity_array==c] for c in indices])
print(pcd_class_objects)
print(f'created {len(pcd_class_objects)} unique point clouds, one for every predicted class')

[None, None, None, None, PointCloud with 2284 points., None, None, None, PointCloud with 21123 points., PointCloud with 460333 points., PointCloud with 23091 points., PointCloud with 2923 points., None, PointCloud with 1393333 points.]
created 14 unique point clouds, one for every predicted class


In [ ]:
list(itertools.chain(*[g for g in pcds_clusters if g]))


[PointCloud with 580 points.,
 PointCloud with 399 points.,
 PointCloud with 1599 points.,
 PointCloud with 3775 points.,
 PointCloud with 4410 points.,
 PointCloud with 1951 points.,
 PointCloud with 537 points.,
 PointCloud with 4162 points.,
 PointCloud with 8073 points.,
 PointCloud with 2194 points.,
 PointCloud with 788 points.,
 PointCloud with 276 points.,
 PointCloud with 216 points.,
 PointCloud with 495 points.,
 PointCloud with 242 points.,
 PointCloud with 319 points.,
 PointCloud with 2920 points.,
 PointCloud with 510 points.,
 PointCloud with 207 points.,
 PointCloud with 8156 points.,
 PointCloud with 505 points.,
 PointCloud with 357 points.]

In [ ]:
#(optional)
distance_threshold=0.2
eps=0.2
min_points=100

#filter eerst pcdNode.resource m.b.v. pcd_cad_t
dists = pcdNode.resource.compute_point_cloud_distance(pcd_cad_t) 
ind = np.where(np.asarray(dists) > distance_threshold)[0]
pcd_tls=pcdNode.resource.select_by_index(ind)
pcd_labels=np.int8(las.prediction[ind])
pcds,labels=gmu.split_pcd_by_labels(pcd_tls,pcd_labels)
print(pcds)
print(labels)

#cluster per label
pcds_clusters=[None]*num_classes
labels_clusters=[None]*num_classes
for i,pcd in enumerate(pcds):
    #retrieve clusters
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=100, print_progress=True))    
    pcds,labels=gmu.split_pcd_by_labels(pcd,labels)
    pcds=np.array(pcds)
    #colorize
    t=pcds[labels==-1]
    if t.size>0:
        t[0].paint_uniform_color([0,0,0])
    [g.paint_uniform_color(ut.random_color()) for g in pcds[labels!=-1] if g]    
    pcds_clusters[i]=pcds.tolist()
    labels_clusters[i]=labels
    
    
# pcds_clusters=np.array(pcds_clusters)
# labels_clusters=np.array(labels_clusters)
print(pcds_clusters)
print(labels_clusters)

#visualize
testgeometries=gmu.join_geometries(list(itertools.chain(*[g for g in pcds_clusters if g])))
o3d.visualization.draw_geometries([testgeometries,pcd_cad_t.paint_uniform_color([1,1,1])])  

[PointCloud with 575 points., PointCloud with 324 points., PointCloud with 1232 points., PointCloud with 7050 points., PointCloud with 1377 points., PointCloud with 305 points., PointCloud with 12163 points., PointCloud with 168 points., PointCloud with 657 points., PointCloud with 1930 points., PointCloud with 179 points., PointCloud with 137 points., PointCloud with 4719 points.]
[ 0  1  3  4  5  6  7  8  9 10 11 12 13]
[[PointCloud with 575 points.], [PointCloud with 58 points., PointCloud with 266 points.], [PointCloud with 186 points., PointCloud with 851 points., PointCloud with 195 points.], [PointCloud with 125 points., PointCloud with 6761 points., PointCloud with 164 points.], [PointCloud with 150 points., PointCloud with 520 points., PointCloud with 276 points., PointCloud with 197 points., PointCloud with 234 points.], [PointCloud with 31 points., PointCloud with 171 points., PointCloud with 103 points.], [PointCloud with 238 points., PointCloud with 8976 points., PointClou

In [ ]:
testgeometries=gmu.join_geometries([g.paint_uniform_color(ut.random_color()) for g in pcds])
o3d.visualization.draw_geometries([testgeometries]) 

In [ ]:
# retain points in pcd (pcd_per_class) that are d>t_d of cad (pcd_class_objects) 
distance_threshold=0.1
pcd_per_class_culled=[None]*num_classes
for i,cad,pcd in zip( np.arange(num_classes),pcd_class_objects,pcd_per_class):
    if cad and pcd:
        #compute distance from cad to pcd    
        dists = pcd.compute_point_cloud_distance(pcd_cad_t) #not cad
        #get indices of points outside the distance_threshold
        ind = np.where(np.asarray(dists) > distance_threshold)[0]
        if ind.size >0:
            #compute indices from cad to pcd and retrieve semantic segmenation results
            pcd_per_class_culled[i]=pcd.select_by_index(ind)
print(pcd_per_class_culled)

[None, None, None, None, PointCloud with 8185 points., None, None, None, PointCloud with 216 points., PointCloud with 1056 points., PointCloud with 2920 points., PointCloud with 510 points., None, PointCloud with 9018 points.]


In [ ]:
testgeometries=pcd_per_class_culled
testgeometries=gmu.join_geometries([g.paint_uniform_color(ut.random_color()) for g in pcds[labels!=-1] if g])
o3d.visualization.draw_geometries([testgeometries,pcd_cad_t.paint_uniform_color([1,1,1])])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [ ]:
#db_clustering
import matplotlib.pyplot as plt
pcd=pcd_per_class[4]
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.1, min_points=100, print_progress=True))
pcds,labels=gmu.split_pcd_by_labels(pcd,labels)
print(pcds)
print(labels)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 11
[PointCloud with 2392 points., PointCloud with 5910 points., PointCloud with 301 points., PointCloud with 177 points., PointCloud with 2706 points., PointCloud with 1309 points., PointCloud with 142 points., PointCloud with 190 points., PointCloud with 200 points., PointCloud with 235 points., PointCloud with 113 points., PointCloud with 961 points.]
[-1  0  1  2  3  4  5  6  7  8  9 10]


In [ ]:
pcds=np.array(pcds)
pcds[labels==-1][0].paint_uniform_color([0,0,0])

# testgeometries=pcd_per_class_culled
testgeometries=gmu.join_geometries([g.paint_uniform_color(ut.random_color()) for g in pcds[labels!=-1] if g])
o3d.visualization.draw_geometries([testgeometries,pcds[labels==-1][0].paint_uniform_color([0,0,0]),pcd_cad_t.paint_uniform_color([1,1,1])])  

In [ ]:
n=4
[pcd_class_objects[n],pcd_per_class[n]]

[PointCloud with 2284 points., PointCloud with 14636 points.]

In [ ]:
# testgeometries=gmu.join_geometries([g.paint_uniform_color(ut.random_color()) for g in testgeometries if g])

o3d.visualization.draw_geometries([pcd_class_objects[n],pcd_per_class[n]])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


# CAD vs PCD upscaled

import presence document

In [ ]:
import pandas as pd

df=list(pd.read_csv('D:/Data/2023-03 Elia Substation/not_built.csv').to_numpy()[:,0])
print(df)

['426cdb74-5efa-4b39-b7e3-c23027147f9b', 'b6fdb729-b270-4909-8be2-c60111abe5ed', 'c2215dd6-eb9b-484d-a971-2b792bdeaff9', '04b670b8-87b3-4fa2-880d-b1f95ca0ba92', '05a3222b-8ecf-49ae-9146-7a53cde261a7', '52ee9b0a-8484-4a28-9f27-08d097996fad', '60e1f286-bc0f-4d52-90dc-42e4332cd910', '66db1841-e305-41c0-99eb-2fc6419417f8', '067a12cc-95d1-45ee-94d9-5ead899c065b', '71c00e15-76cb-41ce-becc-11211ca6336b', '83cea352-acb7-4e42-9e0c-c65d15c2fe09', '458c5bbc-15ec-4162-9668-4572de79652d', '856fa639-626a-4d9a-8b9c-9ac5645efc17', '3155d61e-4119-40f5-810a-cabb14beaa92', '3b124718-db1f-40c6-9574-ae8cf8525d36', '3f91f864-60e3-4808-b9a8-3147b0ed5e9f', '04aaab60-c74e-40ae-8306-eac0915e7d03', '4b44cee6-81f0-46e5-8fb1-dc76f103c3f4', '5b935ec6-d37e-4404-94cd-02902a04ac9d', '7f6c498f-65d8-4d9d-a2a6-1dcc87b0364b', '8b4b4b3a-ca33-42a1-b776-883c6ba6b9b7', '23b18e95-4197-47ab-a20d-6bdc4dba48b4', 'a3186217-cc35-4e94-9c5e-05bb89072724', 'aac07016-03f3-4bec-8431-b2c0591688a9', 'ac436888-7f99-4de2-8f55-534229054df8',

import cad

In [ ]:
cad_dir='D:/Data/2023-03 Elia Substation/BLOCKS_ply/'

In [ ]:
cadNodes= []
for f in ut.get_list_of_files(cad_dir):
    #read mesh
    mesh=o3d.io.read_triangle_mesh(f)
    if len(np.asarray(mesh.triangles)) >0:
        #transform to local coordinates
        mesh.transform(transform) 
        
        #assign presence
        presence=0 if ut.get_filename(f) in df else 1
        
        #create cadNodes
        cadNodes.append(MeshNode(name=ut.get_filename(f),
                            resource=mesh,
                            layer=ut.get_folder(f),
                            i=int(ut.get_filename(ut.get_folder(f)).split('_')[0]),
                            presence=presence))
        
print(f' {len(f)} file objects.')
print(f' {len(cadNodes)} cad objects loaded.')
print(f' {len(cadNodes)/len(ut.get_list_of_files(cad_dir))} % of files has valid geometries.')
print(f' {len([n for n in cadNodes if n.presence==1])} geometries are built.')


[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNIN

In [ ]:
print(f' {len([n for n in cadNodes if n.presence==1])} geometries are built.')
print(f' {len([n for n in cadNodes if n.presence==1])/len(cadNodes)} geometries are built.')



 68 geometries are built.
 0.5714285714285714 geometries are built.


In [ ]:
areas = np.array([n.resource.get_surface_area() for n in cadNodes])
print(np.mean(areas))
print(np.where(areas<np.mean(areas))[0])

test=np.array(cadNodes)
smallNodes=test[np.where(areas < np.mean(areas))[0]]
largeNodes=test[np.where(areas > np.mean(areas))[0]]
print(f' {len([n for n in smallNodes if n.presence==1])/len(smallNodes)} % of the smallNodes are built.')
print(f' {len([n for n in largeNodes if n.presence==1])/len(largeNodes)} % of the largeNodes are built.')

# Calculate the number of values to select (10% of the list length)
num_values = int(len(areas) * 0.1)

# Use argpartition to get the indices of the smallest values
indices = np.argpartition(areas, num_values)[:num_values]
smallestNodes=test[indices]
print(f' {len([n for n in smallestNodes if n.presence==1])/len(smallestNodes)} % of the 10% smallest nodes are built.')


2.7015797019773977
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  34  35  36  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 101 102 103 104 105 106 107 108 109 110 111 113 114 115
 118 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 185 187 188 189 190 191 192 193 194 195 196 197 199 200
 201 202 204]
 0.6557377049180327 % of the smallNodes are built.
 0.9545454545454546 % of the largeNodes are built.
 0.35 % of the 10% smallest nodes are built.


import pcd

In [ ]:
las  = laspy.read(pcd_results_path)
pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las,transform=transform,getColors=True))

visualize

In [ ]:
#colorize objects that are not the closest object to the point clouds
[n.resource.paint_uniform_color([1,0,0]) for n in cadNodes if n.presence==0] #red
[n.resource.paint_uniform_color([1,1,1]) for n in cadNodes if n.presence==1] #white
joined_mesh=gmu.join_geometries([n.resource for n in cadNodes])
o3d.visualization.draw_geometries([pcdNode.resource,joined_mesh]) 

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


remove points of pcdNode that lie to far away

In [ ]:
pcd_cad,identity_array=gmu.create_identity_point_cloud([n.resource for n in cadNodes],resolution=0.02)   
# #crop point cloud #! this takes 30s per run, we expect easly 50-100 clusters
dists=pcdNode.resource.compute_point_cloud_distance(pcd_cad)
ind=np.where(np.asarray(dists) < 1)[0]
pcd_tls=pcdNode.resource.select_by_index(ind)
pcd_labels=np.int8(las.label[ind])
pcd_predictions=np.int8(las.prediction[ind])
print(len(np.asarray(pcd_tls.points)))

10701774


locally register pcd_cad and pcdNode

In [ ]:
#group cad objects in clusters
list_of_nodes=cadNodes
clusters=0
#prepare point clouds for registration
voxel_pcd=pcd_tls.voxel_down_sample(0.04)

while list_of_nodes:
    clusters+=1
    print(len(list_of_nodes))
    
    #select nearby nodes
    nodes,_=tl.select_nodes_with_centers_in_radius(list_of_nodes[0],list_of_nodes,10)  
    # give indices
    indices=[i for i,n in enumerate(cadNodes) if n in nodes]
    print(f'number of nearby nodes: {len(nodes)}')
    print(f'indices of nearby nodes: {indices}')
    
    #create point cloud of cad
    resources=[n.resource for n in nodes if n.resource]
    pcd_cad,identity_array=gmu.create_identity_point_cloud(resources,resolution=0.02)   
    
    # #locally register cadNodes if within 1m #! this only takes 4s per run with voxels
    voxel_cad=pcd_cad.voxel_down_sample(0.04)
    # voxel_pcd=pcd_tls.voxel_down_sample(0.04)
    registration = o3d.pipelines.registration.registration_icp(
        voxel_cad, voxel_pcd,max_correspondence_distance =1 )
    if (registration.fitness>0.8 and registration.inlier_rmse<0.1):
        print(f'regsitration: fitness {registration.fitness}, inlier_rmse: {registration.inlier_rmse}')
        [cadNodes[i].resource.transform(registration.transformation) for i in indices] 
        
    #remove nodes from list_of_nodes
    list_of_nodes = [n for n in list_of_nodes if n not in nodes]

2074
number of nearby nodes: 1
indices of nearby nodes: [0]
regsitration: fitness 1.0, inlier_rmse: 0.044667398297235636
2073
number of nearby nodes: 1
indices of nearby nodes: [1]
2072
number of nearby nodes: 19
indices of nearby nodes: [2, 4, 5, 7, 8, 13, 15, 18, 19, 20, 24, 32, 33, 36, 41, 46, 51, 52, 53]
2053
number of nearby nodes: 100
indices of nearby nodes: [3, 6, 11, 12, 14, 16, 17, 25, 26, 27, 28, 31, 34, 37, 40, 44, 45, 47, 49, 50, 60, 61, 62, 67, 68, 70, 77, 78, 80, 81, 84, 88, 90, 92, 101, 104, 105, 111, 112, 113, 115, 122, 123, 127, 129, 133, 137, 139, 143, 149, 156, 157, 160, 163, 164, 165, 167, 169, 170, 172, 173, 175, 176, 181, 184, 187, 188, 189, 192, 194, 201, 202, 203, 207, 210, 213, 214, 215, 216, 219, 221, 223, 224, 227, 233, 241, 245, 250, 255, 259, 262, 267, 270, 276, 279, 282, 285, 289, 298, 299]
1953
number of nearby nodes: 1
indices of nearby nodes: [9]
1952
number of nearby nodes: 1
indices of nearby nodes: [10]
regsitration: fitness 1.0, inlier_rmse: 0.0479

## Evaluate which objects are not in the point cloud i.e. which objects do no have nearby points


sample cad

In [ ]:
#sample point cloud
pcd_cad,identity_array=gmu.create_identity_point_cloud([n.resource for n in cadNodes],resolution=0.02)  

option A: mark objects that do not have pcd points as their closest points 

In [ ]:
#compute distance from pcd to cad
indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcd_tls.points),np.asarray(pcd_cad.points))

In [ ]:
distance_threshold=0.2

#filter distances
inlier_indices=indices[distances[:,0] < distance_threshold]
#get cad indices that lie within that threshold
indices_in_cad=identity_array[inlier_indices]
indices_unique_in_cad=np.unique(indices_in_cad)

#reset cadNodes 
for n in cadNodes:
    n.detection=0
#fill in detection results
for i in np.unique(indices_in_cad):
    cadNodes[i].detection=1

#report detection rate in function of the built elements
true_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==1]) # elements that are actually built and detected
false_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==0])  # elements that are detected but not built
relevant_elements=len([n for n in cadNodes if n.presence ==1 ]) # total built elements
unbuilt_elements=len([n for n in cadNodes if n.presence ==0 ])

print(f'Built elements: {relevant_elements}')    
print(f'Missng elements: {unbuilt_elements}')  
print(f'---------------------')  
print(f'Detected Built elements: {true_positives}')    
print(f'Detected missing elements: {false_positives}')  
print(f'---------------------')  
print(f'Recall: {true_positives/relevant_elements}')
print(f'Precision: {true_positives/(true_positives+false_positives)}')


Built elements: 141
Missng elements: 64
---------------------
Detected Built elements: 134
Detected missing elements: 51
---------------------
Recall: 0.950354609929078
Precision: 0.7243243243243244


visualize

In [ ]:
#colorize objects that are not the closest object to the point clouds
[n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes ] #white
[n.resource.paint_uniform_color([1,0,0]) for n in cadNodes if (n.detection ==1 and n.presence==0 ) ] #red for outliers
[n.resource.paint_uniform_color([0,1,0]) for n in cadNodes if (n.detection ==0 and n.presence==1 )] #green for outliers

joined_mesh=gmu.join_geometries([n.resource for n in cadNodes])
o3d.visualization.draw_geometries([pcd_tls,joined_mesh])  

In [ ]:
# #compute indices from cad to pcd and retrieve semantic segmenation results
# indices,_=gmu.compute_nearest_neighbors(np.asarray(pcd_cad_t.points),np.asarray(pcdNode.resource.points))
# pcd_labels=np.int8(las.label[indices])
# pcd_predictions=np.int8(las.prediction[indices])

option B: mark objects that have at least one point far from the point cloud

In [ ]:
distance_threshold=0.2

#compute distance from cad to pcd
dists = pcd_cad.compute_point_cloud_distance(pcd_tls)

#get indices of points outside the distance_threshold
ind = np.where(np.asarray(dists) > distance_threshold)[0]
indices_not_in_cad=np.unique(identity_array[ind])

#reset cadNodes and fill in new members
for n in cadNodes:
    n.detection=1
for i in indices_not_in_cad:
    cadNodes[i].detection=0

#report detection rate in function of the built elements
true_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==1]) # elements that are actually built and detected
false_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==0])  # elements that are detected but not built
relevant_elements=len([n for n in cadNodes if n.presence ==1 ]) # total built elements
unbuilt_elements=len([n for n in cadNodes if n.presence ==0 ])

print(f'Built elements: {relevant_elements}')    
print(f'Missng elements: {unbuilt_elements}')  
print(f'---------------------')  
print(f'Detected Built elements: {true_positives}')    
print(f'Detected missing elements: {false_positives}')  
print(f'---------------------')  
print(f'Recall: {true_positives/relevant_elements}')
print(f'Precision: {true_positives/(true_positives+false_positives)}')

Built elements: 141
Missng elements: 64
---------------------
Detected Built elements: 110
Detected missing elements: 41
---------------------
Recall: 0.7801418439716312
Precision: 0.7284768211920529


option C: mark objects with not atleast 70% of points near pcd

In [ ]:
distance_threshold=0.2
percentage_threshold=0.1

#compute distance from cad to pcd
dists = pcd_cad.compute_point_cloud_distance(pcd_tls)

#get indices of points outside the distance_threshold
ind = np.where(np.asarray(dists) <= distance_threshold)[0]
indices_unique_in_cad,counts=np.unique(identity_array[ind],return_counts=True)
unique_totals,count_totals=np.unique(identity_array,return_counts=True)
indices_in_cad=[]
for i,c in zip(indices_unique_in_cad,counts):
    if c/count_totals[i]>percentage_threshold:
        indices_in_cad.append(i)

#reset cadNodes 
for n in cadNodes:
    n.detection=0
#fill in detection results
for i in np.unique(indices_in_cad):
    cadNodes[i].detection=1        

#report detection rate in function of the built elements
true_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==1]) # elements that are actually built and detected
false_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==0])  # elements that are detected but not built
relevant_elements=len([n for n in cadNodes if n.presence ==1 ]) # total built elements
unbuilt_elements=len([n for n in cadNodes if n.presence ==0 ])

print(f'Built elements: {relevant_elements}')    
print(f'Missng elements: {unbuilt_elements}')  
print(f'---------------------')  
print(f'Detected Built elements: {true_positives}')    
print(f'Detected missing elements: {false_positives}')  
print(f'---------------------')  
print(f'Recall: {true_positives/relevant_elements}')
print(f'Precision: {true_positives/(true_positives+false_positives)}')

Built elements: 141
Missng elements: 64
---------------------
Detected Built elements: 134
Detected missing elements: 50
---------------------
Recall: 0.950354609929078
Precision: 0.7282608695652174


Option D: semantic segmentation enrichment

In [ ]:
# filter point cloud
ind=np.where(pcd_labels==pcd_predictions)[0]
pcd_seg=pcd_tls.select_by_index(ind)
pcd_seg_predictions=pcd_predictions[ind]

C:\Users\u0094523\AppData\Local\Temp/ipykernel_15812/3701461619.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  ind=np.where(pcd_labels==pcd_predictions)[0]


In [ ]:
#compute distances from cad to pcd_seg
indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcd_cad.points), np.asarray(pcd_seg.points))

In [ ]:
from itertools import islice

distance_threshold=0.2
percentage_threshold=0.3

#reset cadNodes 
for n in cadNodes:
    n.detection=0

# #compute where points are valid
mask = np.where(distances <= distance_threshold,1,0)[:,0] #! can we take object layers?


#determine percentual presence
_,counts=np.unique(identity_array,return_counts=True)
sums=[np.sum(a) for a in [list(islice(mask, i)) for i in counts]]

for i,n in enumerate(cadNodes):
    if sums[i]/counts[i]>=percentage_threshold:
        cadNodes[i].detection=1
        
#report detection rate in function of the built elements
true_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==1]) # elements that are actually built and detected
false_positives=len([n for n in cadNodes if n.detection ==1 and n.presence==0])  # elements that are detected but not built
relevant_elements=len([n for n in cadNodes if n.presence ==1 ]) # total built elements
unbuilt_elements=len([n for n in cadNodes if n.presence ==0 ])

print(f'Built elements: {relevant_elements}')    
print(f'Missng elements: {unbuilt_elements}')  
print(f'---------------------')  
print(f'Detected Built elements: {true_positives}')    
print(f'Detected missing elements: {false_positives}')  
print(f'---------------------')  
print(f'Recall: {true_positives/relevant_elements}')
print(f'Precision: {true_positives/(true_positives+false_positives)}')

Built elements: 1454
Missng elements: 620
---------------------
Detected Built elements: 1316
Detected missing elements: 617
---------------------
Recall: 0.9050894085281981
Precision: 0.6808070356958096


In [ ]:
# colorArray=gmu.array_to_colors(pcd_predictions)
# pcd_tls.colors=o3d.utility.Vector3dVector(colorArray)
#colorize objects that are not the closest object to the point clouds
#colorize objects that are not the closest object to the point clouds
[n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes ] #white
[n.resource.paint_uniform_color([1,0,0]) for n in cadNodes if (n.detection ==1 and n.presence==0 ) ] #red for outliers
[n.resource.paint_uniform_color([0,1,0]) for n in cadNodes if (n.detection ==0 and n.presence==1 )] #green for outliers

joined_mesh=gmu.join_geometries([n.resource for n in cadNodes])
o3d.visualization.draw_geometries([pcd_seg,joined_mesh])  

# CAD vs classified PCD upscaled

Evaluate which objects are not drawn i.e. which point cloud points marked as equipment are not near and drawn objects.

separate point cloud 

In [ ]:
distance_threshold=0.5

#filter pcdNode.resource so it only contains points further than a threshold from pcd_cad
dists = pcdNode.resource.compute_point_cloud_distance(pcd_cad) 
ind = np.where(np.asarray(dists) > distance_threshold)[0]
pcd_tls=pcdNode.resource.select_by_index(ind)
pcd_labels=np.int8(las.prediction[ind])
pcd_classes,pcd_class_labels=gmu.split_pcd_by_labels(pcd_tls,pcd_labels)
print(pcd_classes)
print(pcd_class_labels)

[PointCloud with 80948 points., PointCloud with 49533 points., PointCloud with 509470 points., PointCloud with 176686 points., PointCloud with 108056 points., PointCloud with 370581 points., PointCloud with 408403 points., PointCloud with 74132 points., PointCloud with 107818 points., PointCloud with 132764 points., PointCloud with 78286 points., PointCloud with 28232 points., PointCloud with 62764 points., PointCloud with 278342 points.]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


create clusters

In [ ]:
#segment pcd_tls per class and per cluster
eps=0.2
min_points=500

#cluster per label
pcdNodes=[]
#iterate over pcd_classes
for pcd,layer in zip(pcd_classes,pcd_class_labels):
    
    #compute clusters
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points))  
    clusters,labels=gmu.split_pcd_by_labels(pcd,labels)    
    clusters=np.array(clusters)
    
    #create pointcloudnode per instance (if not -1)
    for c,l in zip(clusters,labels):
        if l!=-1:
            pcdNodes.append(PointCloudNode(resource=c,
                                        layer=layer
                                        ))
print(f' {len(pcdNodes)} new potential nodes are found.')

 370 new potential nodes are found.


In [ ]:

np.unique([n.layer for n in pcdNodes],return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13], dtype=int8),
 array([ 16,   7,  55,  10,  15,  34, 114,  14,  29,  30,  10,   3,  33],
       dtype=int64))

visualize

In [164]:
#colorize per layer
colors=[ut.random_color() for c in np.arange(num_classes)]
# [n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes]

[n.resource.paint_uniform_color(colors[n.layer]) for n in pcdNodes]
joined_pcd=gmu.join_geometries([n.resource for n in pcdNodes])
joined_cad=gmu.join_geometries([n.resource.paint_uniform_color([0,0,0]) for n in cadNodes])

o3d.visualization.draw_geometries([joined_cad,joined_pcd,pcd_tls.paint_uniform_color([0.9,0.9,0.9])])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


create cadNode examples

In [ ]:
#compute distance
indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcd_cad.points), np.asarray(pcdNode.resource.points))

In [ ]:
print(identity_array.size)
print(len(np.asarray(pcd_cad.points)))
print(distances.size)
print(identity_array.size)



18301125
18301125
18301125
18301125


In [ ]:
for i,n in enumerate(cadNodes): #! this is poorly written
    cadNodes[i].pcd=None
    if n.presence==1:        
        # take a slice and compare distances
        ind=indices[identity_array==i ] #& (distances < distance_threshold)       
        
        # ind = indices[(distances < distance_threshold) & ())[0]]
        cadNodes[i].pcd=pcdNode.resource.select_by_index(ind)            
        # print(pcdNode.resource.select_by_index(ind))

In [ ]:
#colorize per layer
colors=[ut.random_color() for c in np.arange(num_classes)]
# [n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes]

[n.pcd.paint_uniform_color(colors[n.i]) for n in cadNodes if n.pcd is not None]
[n.resource.paint_uniform_color(colors[n.i]) for n in cadNodes if n.pcd is not None]

joined_pcd=gmu.join_geometries([n.pcd for n in cadNodes if getattr(n,'pcd',None) is not None])
joined_cad=gmu.join_geometries([n.resource for n in cadNodes if getattr(n,'pcd',None) is not None])

o3d.visualization.draw_geometries([joined_cad,joined_pcd])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


match and create new Nodes

In [ ]:
# compute normals of cad
count_threshold=400
cadNodes_t=[n for n in cadNodes if n.pcd and len(np.asarray(n.pcd.points)) >count_threshold]
[n.pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(30)) for n in cadNodes_t]
for i,n in enumerate(cadNodes_t):
        cadNodes_t[i].pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(n.pcd,o3d.geometry.KDTreeSearchParamKNN(30))



In [ ]:
for i,node in enumerate(pcdNodes):
    pcdNodes[i].resource.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(30))
    pcdNodes[i].pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(node.resource,o3d.geometry.KDTreeSearchParamKNN(30)) 

In [ ]:
extent_threshold=0.2
max_correspondence_distance =1000


newCadNodes=[]
for i,node in enumerate(pcdNodes):
    
    # find cadNodes with the same layer
    nodeCandidates=[n for n in cadNodes_t if n.i==node.layer]
    
    # check if volumes are approximately the same #! volume is not a good indicator, maybe dimension/ratio
    box=node.resource.get_oriented_bounding_box()    
    
    nodeCandidates=[n for n in nodeCandidates if np.abs(n.get_oriented_bounding_box().extent[0]- box.extent[0])<extent_threshold*box.extent[0]]
    
    pcdNodes[i].nodeCandidates=nodeCandidates
    
    # check if FPFH feature matching has enough inliers en good fitness
    if nodeCandidates:
        
        # #estimate normals
        # node.resource.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(30))
        
        # #compute features
        # node.pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(node.resource,o3d.geometry.KDTreeSearchParamKNN(30)) 
    
        #registration
        for c in nodeCandidates:
            
            candidate=copy.deepcopy(c)
            
            result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            candidate.pcd,node.resource , candidate.pcd_fpfh, node.pcd_fpfh, True,
            max_correspondence_distance ,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            3, [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                    0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                    distance_threshold)
            ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
            if result.fitness >0.7 and result.inlier_rmse<0.1:
                
                # # fine registration
                # max_correspondence_distance=0.3
                # result=o3d.pipelines.registration.registration_generalized_icp(candidate.pcd,node.resource,max_correspondence_distance)
                
                # # if so, create new cadNode
                newCadNodes.append(MeshNode(resource=candidate.resource.transform(result.transformation),
                                            layer=node.layer,
                                            fitness=result.fitness,
                                            inlier_rmse=result.inlier_rmse,
                                            isSameAs=c.subject))
                break
    
print(f' {len(newCadNodes)} new cadNodes are created.')
    

 33 new cadNodes are created.


visualize

In [157]:
#colorize per layer
t=10
# colors=[ut.random_color() for c in np.arange(num_classes)]
[n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in pcdNodes]
[n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes_t]

pcdNodes[t].resource.paint_uniform_color([1,0,0]) 
[n.resource.paint_uniform_color([0,1,0]) for n in pcdNodes[t].nodeCandidates]


joined_pcd=gmu.join_geometries([n.resource for n in pcdNodes])
joined_cad=gmu.join_geometries([n.resource for n in pcdNodes[t].nodeCandidates])
# joined_cad=gmu.join_geometries([n.resource for n in nodeCandidates])


o3d.visualization.draw_geometries([joined_cad,joined_pcd])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [159]:
#colorize per layer
colors=[ut.random_color() for c in np.arange(num_classes)]
# [n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in cadNodes]

[n.resource.paint_uniform_color([0.9,0.9,0.9]) for n in pcdNodes]
[n.resource.paint_uniform_color([1,0,0]) for n in newCadNodes]

# for i,n in enumerate(newCadNodes):
#     g=gmu.sample_geometry(newCadNodes[i].resource)
#     newCadNodes[i].pcd=g[0].paint_uniform_color(colors[n.layer]) 


joined_pcd=gmu.join_geometries([n.resource for n in pcdNodes])
joined_cad=gmu.join_geometries([n.resource for n in newCadNodes])

o3d.visualization.draw_geometries([joined_cad,joined_pcd])  

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


save new nodes to folder

In [ ]:
output_dir='D:/Data/2023-03 Elia Substation/output'
[n.save_resource(output_dir) for n in newCadNodes]

    #save new cadNodes in separate folder
[n.s]